# 任务说明
NLP 當中一個簡單的 task —— 語句分類（文本分類）

給定一個語句，判斷他有沒有惡意（負面標 1，正面標 0）

In [1]:
import time
import os

# 处理警告
import warnings
warnings.filterwarnings('ignore')

path_prefix = 'hw4_data/'

# 读入数据
有三個檔案，分別是 training_label.txt、training_nolabel.txt、testing_data.txt

**training_label.txt**：有 label 的 training data（句子配上 0 or 1，+++ +++只是分隔符號，不要理它）

**training_nolabel.txt**：沒有 label 的 training data（只有句子），用來做 semi-supervised learning

**testing_data.txt**：你要判斷 testing data 裡面的句子是 0 or 1

In [2]:
import torch
import numpy as np
import pandas as pd

In [3]:
def load_training_data(path = 'hw4_data/training_label.txt'):
    # 如果是 'training_label.txt'，需要讀取 label，如果是 'training_nolabel.txt'，不需要讀取 label
    x = []
    y = []
    if 'training_label' in path:
        with open(path_prefix + 'training_label.txt', 'r', encoding='utf-8') as f:
            lines = f.readlines() #读取文本所有内容，并且以数列的格式返回结果
            for i in range(len(lines)):
                lines[i] = lines[i].strip('\n').split(' ')
                x.append(lines[i][2:])
                y.append(lines[i][0])
        return x, y
    else:
        with open(path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            for i in range(len(lines)):
                lines[i] = lines[i].strip('\n').split(' ')
                x.append(lines[i])
        return x

def load_testing_data(path = 'hw4_data/testing_data.txt'): # 把 testing 時需要的 data 讀進來
    X = []
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for i in range(1,len(lines)):
            lines[i] = lines[i].strip('\n').split(',')[1:] #只要“，”后面的部分，返回的是一个数组
            lines[i] = lines[i][0].split(' ') # 按空格分开
            X.append(lines[i])
    return X

# 训练 Word to Vector 模型
訓練 word to vector 的 word embedding，用 cpu，可能要花到 10 分鐘以上

word2vec 主要思想是：一个词的上下文可以很好的表达出词的语义，它是一种通过无监督的学习文本来用产生词向量的方式

word2vec中有两个非常经典的模型：
- **skip-gram**：已知中心词预测周围词。
- **cbow**：已知周围词预测中心词。

**gensim.models.word2vec()**
- **size**: 主要是用来设置神经网络的层数，Word2Vec 中的默认值是设置为 100 层
- **window**: 句子中当前词与目标词之间的最大距离，3表示在目标词前看3 - b个词，后面看 b 个词（ b 在 0 - 3 之间随机）
- **min_count**: 对词进行过滤，频率小于 min-count 的单词则会被忽视，默认值为5
- **workers**: 设置并发训练时候的线程数，不过仅当 Cython 安装的情况下才会起作用, 否则只能使用单核
- **iter**：随机梯度下降法中迭代的最大次数，默认是 5
- **sg**: 默认 sg = 0 为 CBOW 算法; sg = 1为 skip-gram 算法，对低频词敏感

In [19]:
import argparse
from gensim.models import Word2Vec

def train_word2vec(x): 
    model = word2vec.Word2Vec(x, size=250, window=5, min_count=5, workers=4, iter=10, sg=1)
    return model

## 加载数据

In [6]:
# 加载训练数据
print("loading training data ...")
train_x, y = load_training_data(path_prefix + 'training_label.txt')
train_x_no_label = load_training_data(path_prefix + 'training_nolabel.txt')

# 加载测试数据
print("loading testing data ...")
test_x = load_testing_data(path_prefix + 'testing_data.txt')

loading training data ...
loading testing data ...


## 训练

In [7]:
# 训练
print("训练label数据中...")
start_time = time.time()
model = train_word2vec(train_x + test_x)
print("训练用时：%.2f S" % (time.time() - start_time))

#保存训练好的模型
model.save(path_prefix + 'w2v_label_only.model')

'''
# 训练
print("训练所有数据中...")
start_time = time.time()
model = train_word2vec(train_x + train_x_no_label + test_x)
print("训练用时：%.2f S" % (time.time() - start_time))

# 保存训练好的模型
model.save(path_prefix + 'w2v_all.model'))
'''

训练label数据中...
训练用时：89.07 S


'\n# 训练\nprint("训练所有数据中...")\nstart_time = time.time()\nmodel = train_word2vec(train_x + train_x_no_label + test_x)\nprint("训练用时：%.2f S" % (time.time() - start_time))\n\n# 保存训练好的模型\nmodel.save(path_prefix + \'w2v_all.model\'))\n'

# 数据预处理

In [18]:
from torch import nn

class Preprocess():
    def __init__(self, sentences, sen_len, w2v_path="hw4_data/w2v_label_only.mode"): #默认读入已经label数据
        self.w2v_path = w2v_path
        self.sentences = sentences #句子
        self.sen_len = sen_len #句子长度
        self.idx2word = []
        self.word2idx = {}
        self.embedding_matrix = [] # 每行是一个单词对应的向量
    
    def get_w2v_model(self):# 把之前訓練好的 word to vec 模型讀進來        
        self.embedding = Word2Vec.load(self.w2v_path)
        self.embedding_dim = self.embedding.vector_size # 词向量的维数
    
    def add_embedding(self, word): # 用来生成  <PAD> 和 <UNK> 的词向量
        # 把新的 word 加入到已经存在的 embedding_matrix
        # 把 word 加進 embedding，並賦予他一個隨機生成的 representation vector
        # 其中, word 只會是 <PAD> 表示填充以统一长度 或 <UNK> 表示字典中没有的字
        vector = torch.empty(1, self.embedding_dim) #随机生成 word 的向量
        torch.nn.init.uniform_(vector) # vector初始化, 给一组随机值, 均匀分布，默认为(0, 1)
        self.word2idx[word] = len(self.word2idx) # 给word一个编号，根据word在 self.word2idx 中找到编号
        self.idx2word.append(word) # 把word放入数组 idx2word [] 中
        self.embedding_matrix = torch.cat([self.embedding_matrix, vector], 0) #将 vector 拼接到 embedding_matrix 下面
    
    def make_embedding(self, load = True): 
        # 读入或者生成一个 Word2vec word embedding, 并将读入的矩阵按行取出，然后加入到 self.embedding_matrix
        print("Get embedding ...")
        if load:
            print("loading word to vec model ...")
            self.get_w2v_model() # 取得訓練好的 Word2vec word embedding
        else:
            raise NotImplementedError # 预留load != True不实现，要求在其子类中实现，如果直接调用就报一个错误
        for i, word in enumerate(self.embedding.wv.vocab): #model.wv.vocab 可以直接调用生成的词向量
            print('get words #{}'.format(i+1), end='\r')
            self.word2idx[word] = len(self.word2idx) # 製作一個 word2idx 的 dictionary, e.g. self.word2index['he'] = 1 
            self.idx2word.append(word) # 製作一個 idx2word 的 list, e.g. self.index2word[1] = 'he'
            self.embedding_matrix.append(self.embedding[word]) # 製作一個 word2vector 的 list, e.g. self.vectors[1] = 'he' 的vector
        print('')
        self.embedding_matrix = torch.tensor(self.embedding_matrix) # 转成tensor, torch.Tensor是默认tensor类型（torch.FlaotTensor）的简称
        self.add_embedding("<PAD>")# 將 "<PAD>" 加進 embedding_matrix 裡面, 短的句子需要补<PAD>
        self.add_embedding("<UNK>")# 將 "<UNK>" 加進 embedding_matrix 裡面, 没有词向量的词，统一用随机的<UNK>词向量表示
        print("total words: {}".format(len(self.embedding_matrix)))
        return self.embedding_matrix
    
    def pad_sequence(self, sentence):# 將每個句子變成一樣的長度
        if len(sentence) > self.sen_len: #太长的截去头部
            sentence = sentence[:self.sen_len]
        else: # 不够的补 "<PAD>"
            pad_len = self.sen_len - len(sentence)
            for _ in range(pad_len):
                sentence.append(self.word2idx["<PAD>"])
        assert len(sentence) == self.sen_len # len(sentence) 和 self.sen_len 必须相等, 否则报错
        return sentence
    
    def sentence_word2idx(self): # 把句子裡面的字轉成相對應的 index
        sentence_list = [] # 一个训练集里所有句子的词的向量
        for i, sen in enumerate(self.sentences):
            print('sentence count #{}'.format(i+1), end='\r')
            sentence_idx = [] #单个句子里词的向量
            for word in sen:
                if (word in self.word2idx.keys()):
                    sentence_idx.append(self.word2idx[word]) # 查到已有词的向量
                else:
                    sentence_idx.append(self.word2idx["<UNK>"]) # 没有词向量的词，统一用随机的<UNK>词向量表示
            sentence_idx = self.pad_sequence(sentence_idx) # 將每個句子變成一樣的長度
            sentence_list.append(sentence_idx)
        return torch.LongTensor(sentence_list) # 转换成 LongTensor 类型, 并返回
    
    def labels_to_tensor(self, y): # 把 labels 轉成 tensor
        y = [int(label) for label in y]
        return torch.LongTensor(y)

# 数据集
把 data 做成 dataset 供 dataloader 取用

實作了 dataset 所需要的 'init', 'getitem', 'len' 好讓 dataloader 能使用

In [30]:
from torch.utils import data

class TwitterDataset(data.Dataset): # Expected data shape like:(data_num, data_len)，a list of numpy array or a list of lists
    def __init__(self, X, y): # input data shape : (data_num, seq_len, feature_dim)
        self.data = X
        self.label = y
    def __getitem__(self, idx):
        if self.label is None: return self.data[idx] # 没有 label 就只返回数据的编号
        return self.data[idx], self.label[idx] # 有 label 就返回数据的编号和 label 的编号
    def __len__(self): # 数据集里的样本个数
        return len(self.data)

# 模型
用来训练的模型：定义一个简单的只有一层的LSTM，其中词向量由gensim训练得到的数据导入

**class LSTM_Net(nn.Module)** 参数
- **embedding_dim**: 输入维数, 一个词向量的维度
- **hidden_dim**: 输出维数
- **num_layers**: LSTM层数，默认是1
- **dropout**: 表示除了最后一层之外都引入一个dropout
- **bidirectional**: 表示双向LSTM，也就是序列从左往右算一次，从右往左又算一次，这样就可以两倍的输出
- **fix_embedding**: 是否將 embedding fix 固定住，如果 fix_embedding 為 False，在訓練過程中，embedding 也會跟著被訓練
- **batch_first**: 输入输出的第一维是否为 batch_size，默认值 False。 Torch 中，dataloader向神经网络模型连续输入数据，batch_size 参数，表示一批输入多少个数据。 在 LSTM 模型中，输入数据必须是一批数据，为了区分LSTM中的批量数据和dataloader中的批量数据是否相同意义，LSTM 模型就通过这个参数的设定来区分。 如果是相同意义的，就设置为True，如果不同意义的，设置为False。 torch.LSTM 中 batch_size 维度默认是放在第二维度，故此参数设置可以将 batch_size 放在第一维度。如：input 默认是(4,1,5)，中间的 1 是 batch_size，指定batch_first=True后就是(1,4,5)。所以，如果你的输入数据是二维数据的话，就应该将 batch_first 设置为True;

**torch.nn.Embedding.(num_embeddings，Embedding_dim)**：
- num_embeddings: 字典大小，即一个字典里要有多少个词
- embedding_dim: 每个词向量的大小

**torch.nn.Parameter(embedding)**: 将一个不可训练的Tensor转换成可以训练的parameter, 并将这个parameter绑定到这个module里面

In [10]:
class LSTM_Net(nn.Module):
    def __init__(self, embedding, embedding_dim, hidden_dim, num_layers, dropout=0.5, fix_embedding = True):
        super(LSTM_Net, self).__init__()
        # 製作 embedding layer
        self.embedding = torch.nn.Embedding(embedding.size(0),embedding.size(1)) # 生成一个词向量矩阵
        self.embedding.weight = torch.nn.Parameter(embedding)  # 将读出来的模型转换数据类型，并放入词向量矩阵中
        self.embedding.weight.requires_grad = False if fix_embedding else True # 是否將 embedding fix 固定住
        self.embedding_dim = embedding.size(1)
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first = True)
        self.classifier = nn.Sequential( nn.Dropout(dropout), nn.Linear(hidden_dim, 1), nn.Sigmoid() ) # 网络结构
    
    def forward(self, inputs):
        inputs = self.embedding(inputs) # 先将索引映射为词向量
        x, _ = self.lstm(inputs, None)
        x = x[:, -1, :] # x 的 dimension (batch, seq_len, hidden_size) 取句子最后一个单词输出的 hidden state 丢到分类器中
        x = self.classifier(x)
        return x

# 训练函数
**torch.eq(outputs, labels)**：逐个比较两个向量里相同维度的值是否相等

**torch.item()**: 仅针对向量里只有一个元素的时候使用, 获得向量里的数

**二元交叉熵**：$ L = (l_1 ......l_N) $ ,   $ l_n = -w_n [y_n \ln y_{predict} + (1- y_n) (1 - \ln y_{predict})] $

In [11]:
import torch.optim as optim
import torch.nn.functional as F

def evaluation(outputs, labels): # outputs => probability (float)； labels => labels
    outputs[outputs >= 0.5] = 1 # 大於等於 0.5 為正面
    outputs[outputs < 0.5] = 0 # 小於 0.5 為負面
    correct = torch.sum(torch.eq(outputs, labels)).item() #算出两个向量之间有多少个维度的值相同
    return correct

def training(batch_size, n_epoch, lr, model_dir, train, valid, model, device):
    total = sum(p.numel() for p in model.parameters()) # 获取model.parameters()中一共包含多少个维度
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad) # 计算需要算倒数的 weight 的个数, 不是所有 weight 都需要算倒数
    print('\nstart training, 全部参数个数:{}, 需要训练参数个数:{}\n'.format(total, trainable))
    
    model.train() # 將 model 的模式設為 train，這樣 optimizer 就可以更新 model 的參數
    criterion = nn.BCELoss() # 定義損失函數，這裡我們使用 binary cross entropy loss (二元交叉熵)
    t_batch = len(train) # 批数为样本个数，一批训练一句
    v_batch = len(valid) 
    optimizer = optim.Adam(model.parameters(), lr=lr) # 將模型的參數給 optimizer，並給予適當的 learning rate
    total_loss, total_acc, best_acc = 0, 0, 0 #初始化 loss 和 准确率
    for epoch in range(n_epoch):
        total_loss, total_acc = 0, 0
        for i, (inputs, labels) in enumerate(train): # 這段做 training
            inputs = inputs.to(device, dtype=torch.long) # device 為 "cuda"，將 inputs 轉成 torch.cuda.LongTensor
            labels = labels.to(device, dtype=torch.float) # device為 "cuda"，將 labels 轉成 torch.cuda.FloatTensor，因為等等要餵進 criterion，所以型態要是 float
            optimizer.zero_grad() # 由於 loss.backward() 的 gradient 會累加，所以每次餵完一個 batch 後需要歸零，每句归零一次
            outputs = model(inputs) # 將 input 餵給模型
            outputs = outputs.squeeze() # 去掉最外面的 dimension，好讓 outputs 可以餵進 criterion()
            loss = criterion(outputs, labels) # 計算此時模型的 training loss
            loss.backward() # 算 loss 的 gradient
            optimizer.step() # 更新訓練模型的參數
            correct = evaluation(outputs, labels) # 計算此時模型的 training accuracy
            total_acc += (correct / batch_size)
            total_loss += loss.item()
            print('[ Epoch{}: {}/{} ] loss:{:.3f} acc:{:.3f} '.format(epoch+1, i+1, t_batch, loss.item(), correct*100/batch_size), end='\r')
        print('\nTrain | Loss:{:.5f} Acc: {:.3f}'.format(total_loss/t_batch, total_acc/t_batch*100))

        # 這段做 validation
        model.eval() # 將 model 的模式設為 eval，這樣 model 的參數就會固定住
        with torch.no_grad():
            total_loss, total_acc = 0, 0
            for i, (inputs, labels) in enumerate(valid):
                inputs = inputs.to(device, dtype=torch.long) # device 為 "cuda"，將 inputs 轉成 torch.cuda.LongTensor
                labels = labels.to(device, dtype=torch.float) # device 為 "cuda"，將 labels 轉成 torch.cuda.FloatTensor，因為等等要餵進 criterion，所以型態要是 float
                outputs = model(inputs) # 將 input 餵給模型
                outputs = outputs.squeeze() # 去掉最外面的 dimension，好讓 outputs 可以餵進 criterion()
                loss = criterion(outputs, labels) # 計算此時模型的 validation loss
                correct = evaluation(outputs, labels) # 計算此時模型的 validation accuracy
                total_acc += (correct / batch_size)
                total_loss += loss.item()

            print("Valid | Loss:{:.5f} Acc: {:.3f} ".format(total_loss/v_batch, total_acc/v_batch*100))
            if total_acc > best_acc:
                # 如果 validation 的結果優於之前所有的結果，就把當下的模型存下來以備之後做預測時使用
                best_acc = total_acc
                #torch.save(model, "{}/val_acc_{:.3f}.model".format(model_dir,total_acc/v_batch*100))
                torch.save(model, "{}ckpt.model".format(model_dir))
                print('saving model with acc {:.3f}'.format(total_acc/v_batch*100))
        print('-----------------------------------------------')
        model.train() # 將 model 的模式設為 train，這樣 optimizer 就可以更新 model 的參數（因為剛剛轉成 eval 模式）

# 测试函数
對 testing_data.txt 做預測

In [12]:
def testing(batch_size, test_loader, model, device):
    model.eval()
    ret_output = []
    with torch.no_grad():
        for i, inputs in enumerate(test_loader):
            inputs = inputs.to(device, dtype=torch.long)
            outputs = model(inputs)
            outputs = outputs.squeeze()
            outputs[outputs >= 0.5] = 1 # 大於等於 0.5 為正面
            outputs[outputs < 0.5] = 0 # 小於 0.5 為負面
            ret_output += outputs.int().tolist()
    
    return ret_output

# Main

In [26]:
from sklearn.model_selection import train_test_split

# 通過 torch.cuda.is_available() 的回傳值進行判斷是否有使用 GPU 的環境，如果有的話 device 就設為 "cuda"，沒有的話就設為 "cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 處理好各個 data 的路徑
train_with_label = os.path.join(path_prefix, 'training_label.txt')
train_no_label = os.path.join(path_prefix, 'training_nolabel.txt')
testing_data = os.path.join(path_prefix, 'testing_data.txt')

w2v_path = os.path.join(path_prefix, 'w2v_label_only.model') # 處理 word to vec model 的路徑

# 定義句子長度、要不要固定 embedding、batch 大小、要訓練幾個 epoch、learning rate 的值、model 的資料夾路徑
sen_len = 20
fix_embedding = True # fix embedding during training
batch_size = 8
epoch = 5
lr = 0.001

model_dir = path_prefix # model directory for checkpoint model

print("loading data ...") # 把 'training_label.txt' 跟 'training_nolabel.txt' 讀進來
train_x, y = load_training_data(train_with_label)
train_x_no_label = load_training_data(train_no_label)

# 對 input 跟 labels 做預處理
preprocess = Preprocess(train_x, sen_len, w2v_path = w2v_path)
embedding = preprocess.make_embedding(load = True)
train_x = preprocess.sentence_word2idx()
y = preprocess.labels_to_tensor(y)

# 製作一個 model 的對象
model = LSTM_Net(embedding, embedding_dim = 250, hidden_dim = 150, num_layers = 1, dropout = 0.5, fix_embedding=fix_embedding)
model = model.to(device) # device為 "cuda"，model 使用 GPU 來訓練（餵進去的 inputs 也需要是 cuda tensor）

# 把 data 分為 training data 跟 validation data（將一部份 training data 拿去當作 validation data）
X_train, X_val, y_train, y_val = train_x[:180000], train_x[180000:], y[:180000], y[180000:]

# 把 data 做成 dataset 供 dataloader 取用
train_dataset = TwitterDataset(X = X_train, y = y_train)
val_dataset = TwitterDataset(X = X_val, y = y_val)

# 把 data 轉成 batch of tensors
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True, num_workers = 0)

val_loader = torch.utils.data.DataLoader(dataset = val_dataset, batch_size = batch_size, shuffle = False, num_workers = 0)

# 開始訓練
training(batch_size, epoch, lr, model_dir, train_loader, val_loader, model, device)

loading data ...
Get embedding ...
loading word to vec model ...
get words #23429
total words: 23431
sentence count #200000
start training, 全部参数个数:6099101, 需要训练参数个数:241351

[ Epoch1: 22500/22500 ] loss:0.388 acc:87.500  
Train | Loss:0.47359 Acc: 77.148
Valid | Loss:0.43791 Acc: 79.665 
saving model with acc 79.665
-----------------------------------------------
[ Epoch2: 22500/22500 ] loss:0.640 acc:62.500  
Train | Loss:0.42954 Acc: 80.028
Valid | Loss:0.42581 Acc: 80.350 
saving model with acc 80.350
-----------------------------------------------
[ Epoch3: 22500/22500 ] loss:0.463 acc:87.500  
Train | Loss:0.40704 Acc: 81.316
Valid | Loss:0.42363 Acc: 80.080 
-----------------------------------------------
[ Epoch4: 22500/22500 ] loss:0.139 acc:100.000 
Train | Loss:0.38475 Acc: 82.488
Valid | Loss:0.41838 Acc: 80.710 
saving model with acc 80.710
-----------------------------------------------
[ Epoch5: 22500/22500 ] loss:0.422 acc:87.500  
Train | Loss:0.36096 Acc: 83.739
Valid |

# 预测 并 保存CSV文件

In [29]:
# 開始測試模型並做預測
print("loading testing data ...")
test_x = load_testing_data(testing_data)
preprocess = Preprocess(test_x, sen_len, w2v_path=w2v_path)
embedding = preprocess.make_embedding(load = True)
test_x = preprocess.sentence_word2idx()
test_dataset = TwitterDataset(X = test_x, y = None)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                            batch_size = batch_size,
                                            shuffle = False,
                                            num_workers = 0)
print('\nload model ...')
model = torch.load(os.path.join(model_dir, 'ckpt.model'))
outputs = testing(batch_size, test_loader, model, device)

# 寫到 csv 檔案供上傳 Kaggle
tmp = pd.DataFrame({"id":[str(i) for i in range(len(test_x))],"label":outputs})
print("save csv ...")
tmp.to_csv(os.path.join(path_prefix, '预测.csv'), index = False)
print("Finish Predicting")

loading testing data ...
Get embedding ...
loading word to vec model ...
get words #23429
total words: 23431
sentence count #200000
load model ...
save csv ...
Finish Predicting
